In [ ]:
import requests
from pymongo import MongoClient
import random
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [ ]:
mutual_citations = []

# Convert citation_pairs to a set for faster lookups
citation_set = set(citation_pairs)

for source, cited in citation_pairs:
    # Check if the reverse (cited, source) exists
    if (cited, source) in citation_set:
        mutual_citations.append((source, cited))

In [ ]:
def remove_duplicate_citations(doi_list):
    for item in doi_list:
        doi = item[0]  # The main DOI
        citation_lists = item[1:]  # All citation lists

        # Flatten the citation lists into a single list
        all_citations = [citation for sublist in citation_lists for citation in sublist]

        # Remove duplicates using a set
        unique_citations = list(set(all_citations))

        # Reconstruct the item with unique citations
        item[1:] = [unique_citations]  # Replace all citation lists with a single list of unique citations

    return doi_list

In [ ]:
cleaned_doi = remove_duplicate_citations(doi_list)

In [ ]:
positive_pairs = []

for source, cited_dois in cleaned_doi:
    for cited in cited_dois:
        positive_pairs.append((source, cited))

print(positive_pairs)

In [ ]:
all_citations = []
for source, citations in cleaned_doi:
        for i in range(len(citations)):
                all_citations.append(citations[i])

In [ ]:
negative_examples = []

for source, citation in cleaned_doi:
    negative_examples.append([source])
    choices = []
    for i in range(0, 10):
        choice = random.choice(all_citations)
        while choice in citation:
            choice = random.choice(all_citations)
        choices.append(choice)
    negative_examples[-1].append(choices)


In [ ]:
negative_pairs = []
for base, negative in negative_examples:
    for negatives in negative:
        negative_pairs.append((base, negatives))


In [ ]:
training_data = []
for positive in positive_pairs:
    training_data.append(InputExample(texts=[positive[0],positive[1]], label= 1))
for negative in negative_pairs:
    training_data.append(InputExample(texts=[negative[0],negative[1]], label= 0))

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to('cuda')

In [ ]:
train_dataloader = DataLoader(training_data, shuffle= True, batch_size= 32)
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs= 5, warmup_steps=100, optimizer_params= {"lr": 2e-5}, output_path="./fine_tuned", show_progress_bar= True)

In [ ]:
model = SentenceTransformer("./fine_tuned")